In [24]:
import csv
import json
import re
import os
import pandas as pd
from pandas.io.json import json_normalize
import socket
import ssl
from urllib.request import urlopen, Request, URLError
from urllib.parse import quote
from urllib.parse import quote, urlsplit, urlunsplit
from tqdm import tqdm_notebook, tqdm
from random import randint
from time import sleep
from re import sub
pd.set_option('display.max_columns', None)

In [28]:
def get_start_index(path, directory):
    os.chdir(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    os.chdir(path + '/' + directory)
    return len(os.listdir())
def get_json(url_page, headers, data, timeout):
    flag = True
    while flag:
        try:
            if data == None:
                request = Request(url_page, headers=headers)
            else:
                request = Request(url_page, headers=headers, data=data)
            context = ssl._create_unverified_context()
            response = urlopen(request, context=context, timeout=timeout)
            content =  response.read().decode(response.headers.get_content_charset())
            content.replace('\\', '')
            flag = False
        except URLError as e:
            print(type(e))
            flag = True
        except socket.timeout as e:
            print(type(e))
            flag = True
    return json.loads(content)
def get_dataframe(path, directory):
    os.chdir(path + '/' + directory)
    files = os.listdir()
    all_jsons = []
    for file_load in tqdm_notebook(files):
        with open(file_load) as file:
            data_json = json.load(file)
        for x in data_json:
            x['search_postalCode'] = sub(r'\D', '', file_load)
        all_jsons.extend(data_json)
    print('dataframe json_normalizing...')
    df = pd.DataFrame(json_normalize(all_jsons, sep='_'))
    df = df.reset_index()
    del df['index']
    os.chdir(path)
    return df
def get_page_indexes(path, filename):
    df = pd.read_csv(path + '/' + filename)
    return [str(x) for x in df[df.columns[0]]]

In [31]:
def main():
    min_time_sleep = 1
    max_time_sleep = min_time_sleep + 4
    timeout = 10
    path = 'C:/Users/vgarshin/Documents/Py/pochta'
    directory = 'batches_load'
    filename = 'postoffices_indexes_20190306.csv'
    page_start = get_start_index(path, directory)
    post_indexes = get_page_indexes(path, filename)
    print('loaded indexes: ', len(post_indexes))
    print('first index: ', post_indexes[0])
    print('last index: ', post_indexes[-1])
    url_page = 'https://www.pochta.ru/suggestions/v2/postoffice.find-nearest-by-postalcode-raw-filters'
    headers = {'content-type': 'application/json',
               'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.99 YaBrowser/19.1.1.909 Yowser/2.5 Safari/537.36'}
    print('total elements: ', len(post_indexes))
    print('start page: ', page_start)
    for post_index in tqdm_notebook(post_indexes[page_start:]):
        data = '{"postalCode":' + str(post_index) + ',"limit":10,"radius":100,"offset":0,"filters":[],"currentDateTime":"2019-02-28T10:44:32"}'
        data = data.encode()
        data_post_offices = get_json(url_page, headers, data, timeout)
        filename = 'batches_load_' + str(post_index) + '.txt'
        with open(filename, 'w') as file:
            json.dump(data_post_offices, file)
        sleep(randint(min_time_sleep, max_time_sleep))
    print('dataframe creating...')
    df = get_dataframe(path, directory)
    os.chdir(path)
    print('dataframe saving...')
    df.to_csv('data_postoffices_20190306.csv', sep='\t', encoding='cp1251')
    print('data frame created of shape: ', df.shape)
    print(df.columns)

In [32]:
if __name__ == '__main__':
    main()

loaded indexes:  29567
first index:  606130
last index:  690066
total elements:  29567
start page:  29567


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


dataframe creating...


HBox(children=(IntProgress(value=0, max=29567), HTML(value='')))


dataframe saving...
data frame created of shape:  (292496, 19)
Index(['addressSource', 'availability_servicesConclusion',
       'availability_servicesMail', 'availability_servicesPhone', 'holidays',
       'isClosed', 'isTemporaryClosed', 'latitude', 'longitude', 'phones',
       'postalCode', 'search_postalCode', 'serviceGroups', 'settlement',
       'temporaryAddressSource', 'temporaryClosedReason', 'typeCode', 'typeId',
       'workingHours'],
      dtype='object')


In [33]:
df = pd.read_csv('data_postoffices_20190306.csv', sep='\t', encoding='cp1251')
df.head()

,Unnamed: 0,addressSource,availability_servicesConclusion,availability_servicesMail,availability_servicesPhone,holidays,isClosed,isTemporaryClosed,latitude,longitude,phones,postalCode,search_postalCode,serviceGroups,settlement,temporaryAddressSource,temporaryClosedReason,typeCode,typeId,workingHours
0,0,"Мясницкая ул, 26",CONDITIONALLY,client@russianpost.ru,88002005888,"[{'beginWorkTime': '00:00:00', 'date': '2019-0...",False,False,55.763938,37.637281,"[{'phoneIsFax': False, 'phoneNumber': '1000000...",101000,101000,"[{'level': 0, 'serviceCode': '3000000', 'servi...",Москва,NaN,NaN,ГОПС,0,"[{'beginWorkTime': '00:00:00', 'endWorkTime': ..."
1,1,"Мясницкая ул, 26",CONDITIONALLY,client@russianpost.ru,88002005888,"[{'date': '2019-03-07', 'weekDayId': 4, 'weekD...",False,True,55.763938,37.637281,"[{'phoneIsFax': False, 'phoneNumber': '1000000...",105062,101000,"[{'level': 0, 'serviceCode': '1000000', 'servi...",Москва,NaN,Нерентабельность,ГОПС,0,NaN
2,2,"Мясницкая ул, 26",NaN,NaN,NaN,[],False,False,55.763938,37.637281,"[{'phoneIsFax': False, 'phoneNumber': '6231888...",101700,101000,"[{'level': 0, 'serviceCode': '3000000', 'servi...",Москва,NaN,NaN,NaN,0,"[{'beginWorkTime': '09:00:00', 'endWorkTime': ..."
3,3,"Мясницкая ул, 26",NaN,NaN,NaN,[],False,False,55.763938,37.637281,"[{'phoneIsFax': False, 'phoneNumber': '9249628...",101990,101000,[],Москва,NaN,NaN,ГСП,0,"[{'beginWorkTime': '08:00:00', 'endWorkTime': ..."
4,4,"Мясницкая ул, 26",NaN,NaN,NaN,[],False,False,55.763938,37.637281,[],990500,101000,[],Москва,NaN,NaN,ПОЧТОМАТ,0,"[{'beginWorkTime': '00:00:00', 'endWorkTime': ..."
